In [2]:
import os, math, subprocess
import datetime as dt

#data wrangling
import numpy as np
import pandas as pd

#visualization
import seaborn as sns
import matplotlib.pyplot as plt

#excel library
import xlrd

In [3]:
#Loading data
CA = pd.read_excel('KPMG.xlsx', sheet_name= 'CustomerAddress')
CA.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


Check missing values

In [4]:
CA.info()
#no null

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
customer_id           3999 non-null int64
address               3999 non-null object
postcode              3999 non-null int64
state                 3999 non-null object
country               3999 non-null object
property_valuation    3999 non-null int64
dtypes: int64(3), object(3)
memory usage: 187.5+ KB


Customer ID

In [5]:
#check customer id in each sheets:
CA['customer_id'].drop_duplicates().count()

3999

Address

In [6]:
#Duplication
CA.duplicated(subset=['address']).sum()

3

In [7]:
#Duplication list
CA_dup = CA[CA.duplicated(['address'],keep=False)]
CA_dup

,customer_id,address,postcode,state,country,property_valuation
732,737,3 Talisman Place,4811,QLD,Australia,2
2315,2320,64 Macpherson Junction,2208,NSW,Australia,11
2328,2333,3 Mariners Cove Terrace,3108,VIC,Australia,10
2470,2475,3 Talisman Place,4017,QLD,Australia,5
2980,2985,3 Mariners Cove Terrace,2216,NSW,Australia,10
3535,3540,64 Macpherson Junction,4061,QLD,Australia,8


In [8]:
#There is an issue of addresses starting with 0
sort_address = CA.sort_values('address')
sort_address['address'].head()

2020                0 3rd Road
1211    0 American Ash Parkway
3859          0 Arapahoe Court
187               0 Bay Center
2247            0 Blaine Point
Name: address, dtype: object

In [9]:
#Fix address number = 0 and modify address number has 0 before it.
CA['address'] = CA['address'].str.strip("0")
CA['address'].sort_values().head()
#How do I list out the addresses without street number?

3476                1st Trail
2020                 3rd Road
288                  5th Lane
842           Algoma Crossing
1211     American Ash Parkway
Name: address, dtype: object

In [17]:
#Convert ordinal number to cardinal number
CA['address'] = CA['address'].str.replace(r"(\d+)(st|nd|rd|th)\b", r"\1")
CA['address'] = CA['address'].str.replace(r'(\d)\s+(\d)', r'\1\2')
CA['address'].sort_values().tail(20)
#There are problems with addresses having cardinal Number (1st, 3rd, 5th)

652       9912 Eagle Crest Lane
3050         99142 Eastwood Way
147         992 Old Shore Trail
2817        99216 Oakridge Hill
3767         9932 Atwood Center
3844      99476 Mitchell Street
2501    99488 Mccormick Terrace
2088              9953 Kim Lane
2689        99565 Warrior Alley
1016           996 Cody Terrace
1569        996 Morning Parkway
2433    99739 Coolidge Crossing
377     99826 Steensland Avenue
729          9984 Pankratz Road
1828      99841 Talisman Street
1324        9985 Barnett Circle
921           9987 Stuart Alley
1287             9989 Toban Way
532             999 Huxley Hill
1059      9993 Mccormick Street
Name: address, dtype: object

Postcode and State

In [11]:
CA['state'].unique()

array(['New South Wales', 'QLD', 'VIC', 'NSW', 'Victoria'], dtype=object)

In [12]:
#There are 5 inputs:'New South Wales', 'QLD', 'VIC', 'NSW', 'Victoria'
CA['state'] = CA['state'].replace(['New South Wales'], 'NSW')
CA['state'] = CA['state'].replace('Victoria','VIC')
CA['state'].unique()

array(['NSW', 'QLD', 'VIC'], dtype=object)

In [13]:
#dùng groupby filter state tìm min max của postcode trong từng state
test1= CA.groupby(['state'])['postcode'].agg(['min', 'max', 'count','sum'])
test1.head()

,min,max,count,sum
state,,,,
NSW,2000,2880,2140,4953164
QLD,4000,4883,838,3629467
VIC,3000,3995,1021,3357407


property_valuation

In [14]:
test2= CA.groupby(['property_valuation','state'])['postcode'].agg(['min', 'max', 'count','sum'])
test2

min   max  count     sum
property_valuation state                           
1                  NSW    2199  2880     64  174258
                   QLD    4207  4870     48  218967
                   VIC    3012  3860     42  149649
2                  NSW    2340  2870     38   97315
                   QLD    4005  4873     66  301404
                   VIC    3030  3995     39  138646
3                  NSW    2250  2871     48  118736
                   QLD    4012  4883     97  429549
                   VIC    3030  3995     41  140765
4                  NSW    2165  2870     67  166753
                   QLD    4007  4878     94  414011
                   VIC    3011  3977     53  179575
5                  NSW    2143  2870     71  170614
                   QLD    4005  4878     91  396268
                   VIC    3006  3977     63  209173
6                  NSW    2011  2798     75  180583
                   QLD    4012  4879     80  345610
                   VIC    3000  3978     83  281295
7                  NSW    2010  2871    190  458614
                   QLD    4000  4870    148  626406
                   VIC    3000  3977    155  514385
8                  NSW    2000  2780    357  870508
                   QLD    4005  4870    115  484003
                   VIC    3000  3977    174  566432
9                  NSW    2000  2777    432  997380
                   QLD    4020  4870     70  292227
                   VIC    3004  3941    145  464284
10                 NSW    2000  2773    411  905038
                   QLD    4011  4551     22   92240
                   VIC    3008  3977    144  454093
11                 NSW    2000  2756    227  480670
                   QLD    4011  4218      5   20770
                   VIC    3040  3934     49  155774
12                 NSW    2000  2260    160  332695
                   QLD    4005  4007      2    8012
                   VIC    3065  3207     33  103336

# Relevancy

In [18]:
#http://www.sql-join.com/sql-join-types